# Learning 8: Multi-Scenario Comparison

**Duration**: 40 minutes  
**Level**: Advanced  
**Prerequisites**: [Learning 1-3](learning_01_resolution_fundamentals.ipynb), [Learning 4-5](learning_04_gsd_basics.ipynb)

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. **Compare** multiple scenarios systematically within and across domains
2. **Understand** resolution vs coverage tradeoffs for both microscopy and drone imaging
3. **Select** the appropriate scenario preset for a given application
4. **Validate** cross-scenario consistency using quality metrics

---

## Overview

This notebook provides a comprehensive comparison framework for SPIDS scenarios:

| Part | Domain | Focus | Duration |
|------|--------|-------|----------|
| **A** | Microscopy | Objective Shootout (10x/40x/100x) | 15 min |
| **B** | Drone | Altitude Comparison (10m/50m/100m) | 15 min |
| **C** | Cross-Domain | Microscope vs Drone Scale | 10 min |

The goal is to help you make informed decisions about which preset to use for your specific application.

## Setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

# SPIDS imports
from prism.core import create_checkerboard_target, create_usaf_target
from prism.core.instruments import create_instrument
from prism.scenarios import get_scenario_preset, list_scenario_presets
from prism.utils.metrics import compute_ssim, psnr


# Plotting style
plt.rcParams["figure.figsize"] = (14, 8)
plt.rcParams["font.size"] = 11

print("Setup complete!")
print(f"Available microscope presets: {len(list_scenario_presets('microscope'))}")
print(f"Available drone presets: {len(list_scenario_presets('drone'))}")

---

# Part A: Microscope Objective Shootout (15 min)

Let's compare three representative microscope objectives that span the resolution range:

| Objective | NA | Immersion | Best For |
|-----------|----|-----------|---------|
| **10x/0.3** | 0.3 | Air | Sample overview, large features |
| **40x/0.9** | 0.9 | Air | Cellular details, bacteria |
| **100x/1.4** | 1.4 | Oil | Subcellular structures, organelles |

We'll image the same USAF-1951 test target with each objective and compare:
- Resolution achieved
- Field of view
- Image quality (SSIM)

In [ ]:
# Load three microscope presets for comparison
microscope_presets = [
    "microscope_10x_air",
    "microscope_40x_air",
    "microscope_100x_oil",
]

micro_scenarios = {}
microscopes = {}

print("Microscope Objective Comparison")
print("=" * 90)

for preset in microscope_presets:
    scenario = get_scenario_preset(preset)
    micro_scenarios[preset] = scenario

    # Create instrument
    config = scenario.to_instrument_config()
    microscope = create_instrument(config)
    microscopes[preset] = microscope

# Create comparison table
print(f"\n{'Parameter':<25}", end="")
for preset in microscope_presets:
    short_name = preset.replace("microscope_", "").replace("_", " ").title()
    print(f"{short_name:>20}", end="")
print()
print("-" * 85)

params = [
    ("Magnification", lambda s: f"{s._obj.magnification:.0f}x"),
    ("Numerical Aperture", lambda s: f"{s._obj.numerical_aperture:.2f}"),
    ("Immersion Medium", lambda s: s._obj.immersion_medium.capitalize()),
    ("Medium Index (n)", lambda s: f"{s._obj.medium_index:.2f}"),
    ("Lateral Resolution", lambda s: f"{s.lateral_resolution_nm:.0f} nm"),
    ("Axial Resolution", lambda s: f"{s.axial_resolution_um:.2f} µm"),
    ("Field of View", lambda s: f"{s.field_of_view_um:.1f} µm"),
]

for param_name, getter in params:
    print(f"{param_name:<25}", end="")
    for preset in microscope_presets:
        value = getter(micro_scenarios[preset])
        print(f"{value:>20}", end="")
    print()

print("=" * 85)

In [ ]:
# Create USAF-1951 test target
# Use the 40x FOV as reference (middle ground)
ref_scenario = micro_scenarios["microscope_40x_air"]
field_size_m = ref_scenario.field_of_view_um * 1e-6
n_pixels = ref_scenario.n_pixels

# Select USAF groups appropriate for microscopy resolution range
target = create_usaf_target(
    field_size=field_size_m,
    resolution=n_pixels,
    groups=(4, 5, 6, 7),  # High-resolution groups
    margin_ratio=0.2,
)

ground_truth = target.generate()

print("USAF Target created:")
print(f"  Field of view: {field_size_m * 1e6:.1f} µm")
print(f"  Image size: {n_pixels}x{n_pixels} pixels")
print(f"  Pixel size: {field_size_m / n_pixels * 1e9:.1f} nm")
print("  Groups: 4-7 (finest features ~1 µm)")

In [ ]:
# Simulate imaging with all three microscopes
micro_measurements = {}
micro_metrics = {}

input_field = ground_truth.unsqueeze(0).unsqueeze(0)

for preset, microscope in microscopes.items():
    with torch.no_grad():
        measurement = microscope.forward(input_field)

    meas = measurement.squeeze()
    micro_measurements[preset] = meas.cpu().numpy()

    # Compute quality metrics
    gt_norm = (ground_truth - ground_truth.min()) / (ground_truth.max() - ground_truth.min() + 1e-8)
    meas_norm = (meas - meas.min()) / (meas.max() - meas.min() + 1e-8)

    ssim_val = compute_ssim(meas_norm, gt_norm)
    psnr_val = psnr(meas_norm, gt_norm)
    micro_metrics[preset] = {"SSIM": ssim_val, "PSNR": psnr_val}

    scenario = micro_scenarios[preset]
    short_name = preset.replace("microscope_", "")
    print(f"{short_name}: Resolution={scenario.lateral_resolution_nm:.0f}nm, SSIM={ssim_val:.4f}")

print("\nAll microscope simulations complete!")

In [ ]:
# Three-way microscope comparison visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 14))
extent = [0, field_size_m * 1e6, 0, field_size_m * 1e6]  # µm

# Ground truth
axes[0, 0].imshow(ground_truth.cpu().numpy(), cmap="gray", extent=extent, origin="lower")
axes[0, 0].set_title("Ground Truth (USAF-1951)", fontsize=14, fontweight="bold")
axes[0, 0].set_xlabel("Position (µm)")
axes[0, 0].set_ylabel("Position (µm)")

# Microscope measurements
positions = [(0, 1), (1, 0), (1, 1)]
colors = ["blue", "green", "red"]

for (preset, meas), pos, color in zip(micro_measurements.items(), positions, colors):
    ax = axes[pos]
    scenario = micro_scenarios[preset]
    metrics = micro_metrics[preset]

    ax.imshow(meas, cmap="gray", extent=extent, origin="lower")

    short_name = preset.replace("microscope_", "").replace("_", " ").title()
    ax.set_title(
        f"{short_name}\nRes: {scenario.lateral_resolution_nm:.0f} nm | SSIM: {metrics['SSIM']:.3f}",
        fontsize=12,
        color=color,
        fontweight="bold",
    )
    ax.set_xlabel("Position (µm)")
    ax.set_ylabel("Position (µm)")

plt.tight_layout()
plt.show()

print("\nKey Observations:")
print("  • 10x Air: Significant blurring, only coarse features resolved")
print("  • 40x Air: Good resolution, most features visible")
print("  • 100x Oil: Best resolution, finest features clearly resolved")

In [ ]:
# Microscope Objective Decision Guide
print("=" * 80)
print("MICROSCOPE OBJECTIVE DECISION GUIDE")
print("=" * 80)

guide = [
    ("microscope_10x_air", "Sample overview, tissue sections, large specimens", ">1 µm"),
    ("microscope_20x_air", "Tissue details, cell populations, larger bacteria", "~500 nm"),
    ("microscope_40x_air", "Single cells, bacteria, cellular structures", "~370 nm"),
    ("microscope_40x_phase", "Unstained live cells, transparent samples", "~370 nm"),
    ("microscope_40x_dic", "3D-like relief imaging of transparent samples", "~370 nm"),
    ("microscope_60x_water", "Live tissue, aqueous samples, reduced aberration", "~280 nm"),
    ("microscope_100x_oil", "Subcellular organelles, fine detail", "~200 nm"),
    ("microscope_100x_oil_phase", "Unstained subcellular structures", "~200 nm"),
]

print(f"\n{'Preset':<30} {'Best For':<40} {'Resolution'}")
print("-" * 85)
for preset, use_case, res in guide:
    print(f"{preset:<30} {use_case:<40} {res}")
print("=" * 80)

print("\n📌 Quick Selection Rules:")
print("  1. Start with 10x for orientation, then increase magnification")
print("  2. Use Phase/DIC for unstained transparent samples")
print("  3. Use water immersion for live tissue to reduce aberrations")
print("  4. Use oil immersion only when resolution < 300nm is required")
print("  5. Higher NA = better resolution but smaller depth of field")

---

# Part B: Drone Mission Selection (15 min)

Now let's compare drone imaging at three different altitudes:

| Altitude | GSD | Swath | Best For |
|----------|-----|-------|---------|
| **10m** | ~1 cm | ~7 m | Close inspection, fine detail |
| **50m** | ~6.5 cm | ~35 m | Site survey, standard mapping |
| **100m** | ~19 cm | ~65 m | Large area coverage |

We'll image the same scene and compare:
- GSD achieved
- Coverage area (swath width)
- Detail level
- Motion blur effects

In [ ]:
# Load three drone presets for altitude comparison
drone_presets = [
    "drone_10m_inspection",
    "drone_50m_survey",
    "drone_100m_mapping",
]

drone_scenarios = {}
cameras = {}

print("Drone Altitude Comparison")
print("=" * 90)

for preset in drone_presets:
    scenario = get_scenario_preset(preset)
    drone_scenarios[preset] = scenario

    # Create camera instrument
    config = scenario.to_instrument_config()
    camera = create_instrument(config)
    cameras[preset] = camera

# Create comparison table
print(f"\n{'Parameter':<25}", end="")
for preset in drone_presets:
    short_name = f"{drone_scenarios[preset].altitude_m:.0f}m"
    print(f"{short_name:>20}", end="")
print()
print("-" * 85)

params = [
    ("Altitude", lambda s: f"{s.altitude_m:.0f} m"),
    ("GSD", lambda s: f"{s.actual_gsd_cm:.2f} cm"),
    ("Swath Width", lambda s: f"{s.swath_width_m:.1f} m"),
    (
        "Ground Speed",
        lambda s: f"{s.ground_speed_mps:.0f} m/s" if s.ground_speed_mps > 0 else "Hover",
    ),
    ("Motion Blur", lambda s: f"{s.motion_blur_pixels:.2f} px"),
    ("Min Detectable", lambda s: f"{s.actual_gsd_cm * 2:.1f} cm"),  # Nyquist
]

for param_name, getter in params:
    print(f"{param_name:<25}", end="")
    for preset in drone_presets:
        value = getter(drone_scenarios[preset])
        print(f"{value:>20}", end="")
    print()

print("=" * 85)

In [ ]:
# Create checkerboard target for drone comparison
# Use the smallest swath (10m inspection) as field size
ref_drone = drone_scenarios["drone_10m_inspection"]
drone_field_size = ref_drone.swath_width_m
drone_resolution = 1024

# 5cm checkerboard squares - should be:
# - Well resolved at 10m (GSD ~1cm)
# - Marginal at 50m (GSD ~6.5cm)
# - Unresolved at 100m (GSD ~19cm)
square_size = 0.05  # 5 cm

drone_target = create_checkerboard_target(
    field_size=drone_field_size,
    square_size=square_size,
    resolution=drone_resolution,
    margin_ratio=0.25,
)

drone_scene = drone_target.generate()

print("Drone Test Scene created:")
print(f"  Field of view: {drone_field_size:.1f} m")
print(f"  Square size: {square_size * 100:.0f} cm")
print(f"  Image size: {drone_resolution}x{drone_resolution} pixels")
print("\nResolution expectations:")
for preset in drone_presets:
    scenario = drone_scenarios[preset]
    gsd = scenario.actual_gsd_cm
    ratio = (square_size * 100) / gsd  # pixels per square
    status = "RESOLVED" if ratio > 2 else ("MARGINAL" if ratio > 1 else "UNRESOLVED")
    print(f"  {preset}: {ratio:.1f} pixels/square -> {status}")

In [ ]:
# Simulate imaging with all three drone cameras
drone_measurements = {}

input_field = drone_scene.unsqueeze(0).unsqueeze(0)

for preset, camera in cameras.items():
    with torch.no_grad():
        measurement = camera.forward(input_field)

    drone_measurements[preset] = measurement.squeeze().cpu().numpy()
    print(f"Simulated {preset}")

print("\nAll drone simulations complete!")

In [ ]:
# Three-way drone comparison visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 14))
extent = [0, drone_field_size, 0, drone_field_size]  # meters

# Ground truth
axes[0, 0].imshow(drone_scene.cpu().numpy(), cmap="gray", extent=extent, origin="lower")
axes[0, 0].set_title("Ground Truth\n(5cm checkerboard squares)", fontsize=14, fontweight="bold")
axes[0, 0].set_xlabel("Distance (m)")
axes[0, 0].set_ylabel("Distance (m)")

# Drone measurements
positions = [(0, 1), (1, 0), (1, 1)]
colors = ["blue", "green", "red"]

for (preset, meas), pos, color in zip(drone_measurements.items(), positions, colors):
    ax = axes[pos]
    scenario = drone_scenarios[preset]

    ax.imshow(meas, cmap="gray", extent=extent, origin="lower")

    ax.set_title(
        f"{scenario.altitude_m:.0f}m Altitude\nGSD: {scenario.actual_gsd_cm:.1f} cm | Swath: {scenario.swath_width_m:.0f} m",
        fontsize=12,
        color=color,
        fontweight="bold",
    )
    ax.set_xlabel("Distance (m)")
    ax.set_ylabel("Distance (m)")

plt.tight_layout()
plt.show()

print("\nKey Observations:")
print("  • 10m altitude: 5cm squares clearly resolved (GSD << square size)")
print("  • 50m altitude: 5cm squares barely visible (GSD ~ square size)")
print("  • 100m altitude: 5cm squares unresolved (GSD > square size)")

In [ ]:
# All 8 drone presets comparison chart
all_drone_presets = list_scenario_presets("drone")
all_drone_data = []

for name in all_drone_presets:
    scenario = get_scenario_preset(name)
    all_drone_data.append(
        {
            "name": name.replace("drone_", ""),
            "altitude": scenario.altitude_m,
            "gsd": scenario.actual_gsd_cm,
            "swath": scenario.swath_width_m,
            "blur": scenario.motion_blur_pixels,
        }
    )

# Sort by altitude
all_drone_data.sort(key=lambda x: x["altitude"])

# Create visualization
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

names = [d["name"] for d in all_drone_data]
altitudes = [d["altitude"] for d in all_drone_data]
gsds = [d["gsd"] for d in all_drone_data]
swaths = [d["swath"] for d in all_drone_data]
blurs = [d["blur"] for d in all_drone_data]

colors = plt.cm.viridis(np.linspace(0, 0.9, len(all_drone_data)))

# GSD chart
ax1 = axes[0]
bars1 = ax1.bar(range(len(names)), gsds, color=colors, edgecolor="black", linewidth=0.5)
for i, (bar, alt) in enumerate(zip(bars1, altitudes)):
    ax1.annotate(
        f"{alt:.0f}m",
        (bar.get_x() + bar.get_width() / 2, bar.get_height()),
        ha="center",
        va="bottom",
        fontsize=8,
        fontweight="bold",
    )
ax1.set_xticks(range(len(names)))
ax1.set_xticklabels(names, rotation=45, ha="right", fontsize=8)
ax1.set_ylabel("GSD (cm/pixel)")
ax1.set_title("Ground Sampling Distance", fontweight="bold")
ax1.axhline(5, color="green", linestyle="--", alpha=0.7, label="Inspection (<5cm)")
ax1.axhline(10, color="orange", linestyle="--", alpha=0.7, label="Survey (<10cm)")
ax1.legend(loc="upper left", fontsize=8)
ax1.grid(axis="y", alpha=0.3)

# Swath chart
ax2 = axes[1]
ax2.bar(range(len(names)), swaths, color=colors, edgecolor="black", linewidth=0.5)
ax2.set_xticks(range(len(names)))
ax2.set_xticklabels(names, rotation=45, ha="right", fontsize=8)
ax2.set_ylabel("Swath Width (m)")
ax2.set_title("Coverage Width", fontweight="bold")
ax2.grid(axis="y", alpha=0.3)

# GSD vs Swath scatter plot (tradeoff visualization)
ax3 = axes[2]
scatter = ax3.scatter(gsds, swaths, c=altitudes, cmap="viridis", s=150, edgecolor="black")
for i, name in enumerate(names):
    ax3.annotate(
        name.split("_")[0],
        (gsds[i], swaths[i]),
        fontsize=8,
        xytext=(5, 5),
        textcoords="offset points",
    )
ax3.set_xlabel("GSD (cm) - Detail")
ax3.set_ylabel("Swath (m) - Coverage")
ax3.set_title("Resolution vs Coverage Tradeoff", fontweight="bold")
cbar = plt.colorbar(scatter, ax=ax3)
cbar.set_label("Altitude (m)")
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nThe scatter plot shows the fundamental tradeoff:")
print("  • Lower altitude (dark) = Better detail (low GSD) but narrow coverage")
print("  • Higher altitude (light) = Wide coverage but less detail (high GSD)")

In [ ]:
# Drone Mission Planning Decision Guide
print("=" * 90)
print("DRONE MISSION PLANNING GUIDE")
print("=" * 90)

print("\n📋 Mission Type Selection:")
print("-" * 90)

missions = [
    ("Close Inspection", "drone_10m_inspection", "~1 cm", "Cracks, wires, small defects"),
    ("Detailed Survey", "drone_20m_detail", "~1.6 cm", "Structural details, fine features"),
    ("Infrastructure", "drone_infrastructure_30m", "~4 cm", "Buildings, bridges, roads"),
    ("Agriculture", "drone_agriculture_50m", "~5.6 cm", "Crop health, field mapping"),
    ("Site Survey", "drone_50m_survey", "~6.5 cm", "Construction sites, terrain"),
    ("Precision Work", "drone_hover_50m", "~6.5 cm", "Static detailed imaging (no blur)"),
    ("Large Mapping", "drone_100m_mapping", "~19 cm", "Regional coverage, land survey"),
    ("Consumer Drone", "drone_phantom_120m", "~8 cm", "DJI-style general purpose"),
]

print(f"{'Mission Type':<20} {'Preset':<28} {'GSD':<10} {'Can Detect'}")
print("-" * 90)
for mission, preset, gsd, detect in missions:
    print(f"{mission:<20} {preset:<28} {gsd:<10} {detect}")

print("\n" + "=" * 90)

print("\n📌 Quick Selection Rules:")
print("  1. Determine smallest feature you need to detect")
print("  2. GSD should be < half the feature size (Nyquist)")
print("  3. Consider flight time: higher altitude = faster coverage")
print("  4. Use hover mode when motion blur is unacceptable")
print("  5. Legal altitude limits vary by region (often 120m max)")

---

# Part C: Cross-Domain Comparison (10 min)

Now let's put microscopy and drone imaging in perspective by comparing their scales:

| Domain | Resolution Range | Field of View | Applications |
|--------|-----------------|---------------|---------------|
| Microscopy | 200 nm - 1 µm | 50 - 500 µm | Cells, bacteria, materials |
| Drone | 1 cm - 20 cm | 10 - 100 m | Structures, terrain, agriculture |

The scale difference is **~100,000x**!

In [ ]:
# Cross-domain scale comparison
print("=" * 80)
print("CROSS-DOMAIN SCALE COMPARISON")
print("=" * 80)

# Gather all resolution data
all_scenarios = []

# Microscope presets
for preset in list_scenario_presets("microscope"):
    scenario = get_scenario_preset(preset)
    all_scenarios.append(
        {
            "name": preset.replace("microscope_", "µ:"),
            "domain": "Microscope",
            "resolution_m": scenario.lateral_resolution_nm * 1e-9,
            "fov_m": scenario.field_of_view_um * 1e-6,
        }
    )

# Drone presets
for preset in list_scenario_presets("drone"):
    scenario = get_scenario_preset(preset)
    all_scenarios.append(
        {
            "name": preset.replace("drone_", "d:"),
            "domain": "Drone",
            "resolution_m": scenario.actual_gsd_cm * 1e-2,
            "fov_m": scenario.swath_width_m,
        }
    )

# Summary table
print("\n" + "-" * 80)
print(f"{'Domain':<15} {'Resolution Range':<25} {'FOV Range':<25} {'Scale'}")
print("-" * 80)

# Microscope stats
micro_res = [s["resolution_m"] for s in all_scenarios if s["domain"] == "Microscope"]
micro_fov = [s["fov_m"] for s in all_scenarios if s["domain"] == "Microscope"]
print(
    f"{'Microscope':<15} {min(micro_res) * 1e9:.0f} - {max(micro_res) * 1e9:.0f} nm{'':<10} "
    f"{min(micro_fov) * 1e6:.0f} - {max(micro_fov) * 1e6:.0f} µm{'':<10} Nanoscale"
)

# Drone stats
drone_res = [s["resolution_m"] for s in all_scenarios if s["domain"] == "Drone"]
drone_fov = [s["fov_m"] for s in all_scenarios if s["domain"] == "Drone"]
print(
    f"{'Drone':<15} {min(drone_res) * 100:.1f} - {max(drone_res) * 100:.1f} cm{'':<12} "
    f"{min(drone_fov):.0f} - {max(drone_fov):.0f} m{'':<14} Meter scale"
)

print("-" * 80)

# Scale ratio
res_ratio = min(drone_res) / max(micro_res)
fov_ratio = min(drone_fov) / max(micro_fov)
print("\nScale Difference:")
print(f"  Resolution: {res_ratio:.0f}x (drone finest vs microscope coarsest)")
print(f"  Field of View: {fov_ratio:.0f}x (drone smallest vs microscope largest)")

In [ ]:
# Log-scale visualization of all scenarios
fig, ax = plt.subplots(figsize=(14, 8))

# Prepare data
micro_data = [s for s in all_scenarios if s["domain"] == "Microscope"]
drone_data = [s for s in all_scenarios if s["domain"] == "Drone"]

# Plot microscope scenarios
micro_res = [s["resolution_m"] for s in micro_data]
micro_fov = [s["fov_m"] for s in micro_data]
micro_names = [s["name"] for s in micro_data]

ax.scatter(
    micro_res,
    micro_fov,
    s=200,
    c="blue",
    marker="o",
    label="Microscope",
    edgecolor="black",
    alpha=0.7,
)

# Plot drone scenarios
drone_res = [s["resolution_m"] for s in drone_data]
drone_fov = [s["fov_m"] for s in drone_data]
drone_names = [s["name"] for s in drone_data]

ax.scatter(
    drone_res, drone_fov, s=200, c="green", marker="^", label="Drone", edgecolor="black", alpha=0.7
)

# Add annotations for key presets
key_presets = ["µ:100x_oil", "µ:10x_air", "d:10m_inspection", "d:100m_mapping"]
for s in all_scenarios:
    if s["name"] in key_presets:
        ax.annotate(
            s["name"],
            (s["resolution_m"], s["fov_m"]),
            xytext=(10, 10),
            textcoords="offset points",
            fontsize=9,
        )

# Scale lines
ax.axvline(1e-6, color="gray", linestyle="--", alpha=0.5, label="1 µm")
ax.axvline(1e-2, color="gray", linestyle=":", alpha=0.5, label="1 cm")
ax.axhline(1e-3, color="gray", linestyle="--", alpha=0.5)  # 1 mm
ax.axhline(1, color="gray", linestyle=":", alpha=0.5)  # 1 m

ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlabel("Resolution (m) - smaller is better", fontsize=12)
ax.set_ylabel("Field of View (m) - larger is more coverage", fontsize=12)
ax.set_title("SPIDS Scenario Space: Microscopy vs Drone Imaging", fontsize=14, fontweight="bold")
ax.legend(loc="lower right", fontsize=11)
ax.grid(True, alpha=0.3, which="both")

# Add scale annotations
ax.text(1e-7, 5e-5, "Nanoscale\n(nm-µm)", fontsize=10, ha="center", color="blue")
ax.text(1e-1, 50, "Meter scale\n(cm-m)", fontsize=10, ha="center", color="green")

plt.tight_layout()
plt.show()

print("\nThis plot shows the entire SPIDS scenario space:")
print("  • Blue circles: Microscope presets (nanometer resolution, micrometer FOV)")
print("  • Green triangles: Drone presets (centimeter resolution, meter FOV)")
print("  • ~100,000x scale difference between domains!")

In [ ]:
# Application Decision Matrix
print("=" * 90)
print("APPLICATION DECISION MATRIX")
print("=" * 90)

print("\n🔬 USE MICROSCOPY WHEN:")
print("-" * 90)
print("  • Feature size < 100 µm (0.1 mm)")
print("  • Sample fits under objective (typically < 1 cm)")
print("  • Need subcellular or molecular detail")
print("  • Examples: cells, bacteria, materials science, histology")

print("\n🚁 USE DRONE IMAGING WHEN:")
print("-" * 90)
print("  • Feature size > 1 cm")
print("  • Need to cover large areas (> 10 m²)")
print("  • Sample is outdoors or inaccessible")
print("  • Examples: agriculture, construction, infrastructure inspection")

print("\n📊 PRESET SELECTION FLOWCHART:")
print("-" * 90)
print("""
  Feature Size?
  │
  ├─ < 1 µm ──────────► microscope_100x_oil (NA 1.4)
  │
  ├─ 1-10 µm ─────────► microscope_40x_air (NA 0.9)
  │
  ├─ 10-100 µm ───────► microscope_10x_air (NA 0.3)
  │
  ├─ 1-10 cm ─────────► drone_10m_inspection (GSD ~1 cm)
  │
  ├─ 10-50 cm ────────► drone_50m_survey (GSD ~6.5 cm)
  │
  └─ > 50 cm ─────────► drone_100m_mapping (GSD ~19 cm)
""")
print("=" * 90)

---

# Summary

## Key Takeaways

### 1. Microscopy Objectives

| Objective | NA | Resolution | Best For |
|-----------|-----|-----------|----------|
| 10x Air | 0.3 | ~1100 nm | Overview, orientation |
| 40x Air | 0.9 | ~370 nm | Cells, bacteria |
| 100x Oil | 1.4 | ~200 nm | Subcellular detail |

**Rule**: Higher NA = better resolution but smaller depth of field

### 2. Drone Altitudes

| Altitude | GSD | Coverage | Best For |
|----------|-----|----------|----------|
| 10m | ~1 cm | ~7 m | Close inspection |
| 50m | ~6.5 cm | ~35 m | Site survey |
| 100m | ~19 cm | ~65 m | Large area mapping |

**Rule**: Higher altitude = wider coverage but less detail

### 3. Cross-Domain Scaling

- Microscopy: nanometer resolution, micrometer field of view
- Drone: centimeter resolution, meter field of view
- Scale difference: ~100,000x

### 4. Universal Principle

**Resolution vs Coverage Tradeoff** applies to all optical imaging:
- Better resolution = smaller field of view
- Larger coverage = coarser detail

---

## Reference Documentation

- **[Scenario Preset Catalog](../../docs/references/scenario_preset_catalog.md)** - All 17 presets
- **[Optical Resolution Limits](../../docs/references/optical_resolution_limits.md)** - Physics background
- **[Microscopy Parameters](../../docs/references/microscopy_parameters.md)** - Objective specifications
- **[Drone Camera Parameters](../../docs/references/drone_camera_parameters.md)** - Sensor and lens specs

---

## Next Steps

- **[08_custom_scenario_builder.py](../python_api/08_custom_scenario_builder.py)**: Create custom scenarios
- **[05_sampling_density_validation.ipynb](../validation/notebooks/05_sampling_density_validation.ipynb)**: Validate sampling requirements
- **[Learning Path](../../docs/user_guides/learning_path.md)**: Complete curriculum guide

In [ ]:
print("\n" + "=" * 70)
print("Congratulations! You've completed Learning 8: Multi-Scenario Comparison")
print("=" * 70)
print("\nYou now understand:")
print("  [ok] How to compare microscope objectives (10x/40x/100x)")
print("  [ok] How to compare drone altitudes (10m/50m/100m)")
print("  [ok] The resolution vs coverage tradeoff")
print("  [ok] Cross-domain scale relationships (~100,000x)")
print("  [ok] How to select the right preset for your application")
print("\nYou're ready for advanced scenario customization!")